In [2]:
import os
os.chdir('../')

In [5]:
import pandas as pd
from source.lib.helpers import *
import json
import ast
import glob


In [9]:
download_post_date = '2018-09-01'
time_period = 6

In [25]:
df_pypi_github_mapping = pd.read_csv('output/derived/collect_github_repos/linked_pypi_github.csv')
monthly_downloads = pd.read_parquet('drive/output/scrape/pypi_monthly_downloads/pypi_monthly_downloads.parquet')

In [26]:
df_github_downloads = pd.merge(df_pypi_github_mapping.rename(columns={'package':'project'}), monthly_downloads)
df_github_downloads = df_github_downloads[df_github_downloads['github repository'] != 'Unavailable']
df_github_downloads = df_github_downloads[df_github_downloads['month']>=download_post_date]
df_github_downloads = df_github_downloads.rename(columns = {'github repository':'repo_name', 'month': 'created_at'})
df_github_downloads = ImputeTimePeriod(df_github_downloads, time_period)
df_github_downloads = df_github_downloads.groupby(['repo_name', 'time_period','project'])['num_downloads'].sum().reset_index()

df_github = df_github_downloads.groupby(['repo_name', 'time_period']).agg(
    total_downloads=('num_downloads', 'sum'),
    total_downloads_one_project=('num_downloads', 'max'),
    unique_projects=('project', lambda x: set(x)),
    project_count=('project', 'nunique')
).reset_index()
df_github.to_parquet('issue/github_downloads.parquet')

In [ ]:
def flatten_scorecard(row_str):
    if pd.isnull(row_str):
        return
    row = ast.literal_eval(row_str)
    data = {"score": row.get("score")}
    for check in row.get("checks", []):
        name = check.get("name", "").lower().replace("-", "_").replace(" ", "_")
        if name:
            data[f"{name}_name"] = check.get("name")
            data[f"{name}_score"] = check.get("score")
            data[f"{name}_reason"] = check.get("reason")
            data[f"{name}_details"] = check.get("details")
    return data

dfs = []
for file in glob.glob("drive/output/scrape/get_weekly_scorecard_data/scorecard/*.csv"):
    df = pd.read_csv(file).T
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    if 'scorecard_data' in df.columns:
        flat = df["scorecard_data"].apply(flatten_scorecard).apply(pd.Series)
        dfs.append(pd.concat([df, flat], axis=1).assign(source_file=file))
        print(file)
df_scorecard = pd.concat(dfs, ignore_index=True)
df_scorecard.to_parquet('issue/github_scorecard.parquet')

drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_planetlabs_planet-client-python.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_MarketSquare_Robotframework-Database-Library.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_kvesteri_intervals.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_loanzen_falcon-auth.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_mozilla_DeepSpeech.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_avwx-rest_avwx-engine.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_molobrakos_tellduslive.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_wbond_asn1crypto.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_edoburu_django-fluent-dashboard.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/scorecard_bcj_AttrDict.csv
drive/output/scrape/get_weekly_scorecard_data/scorecard/sc

In [ ]:
df_scorecard

In [ ]:
# in terms of outcomes I can do
# average over time period (score)
# whether there was increase/decrease/stability in the outcome over the time period (increase, decrease are individual outcomes - also might want to add increase>=1, decrease>=1)
# could do individual outcomes 